# Import of data, network creation and optimization

In [ ]:
from __future__ import print_function, division
import numpy as np # 1.14.0
import pandas as pd # 0.22
import matplotlib.pyplot as plt # 2.1.2
from datetime import datetime 
import math
import os
import dill as pickle # Transfer data for analysis # dill version 0.2.7.1

import pypsa # 0.13.1
import pypsa.geo as geo

from custom_thesis_functions import *

%matplotlib inline

## Setup

Setting up all the network's main components

All of the data is taken from another thesis, written at the same chair at TU Berlin as this one. The data is provided in the form of excel-files and is read using methods from pandas.

### Variables

In [ ]:
period = pd.date_range(start='2017-01-21 00:00', end='2017-01-21 23:45', freq='h')

In [ ]:
# Variable to add further info to filename about e.g. which uc_restrictions were used
instance_info = ""

In [ ]:
include_fission = False

In [ ]:
marginal_costs_carrier = {"Wind":2,
                          "ee":2,
                          "Kernenergie":3,
                          "Import":10000}

Variables that are to be considered during unit commitment

In [ ]:
uc_restrictions = ["p_nom_min",
              "marginal_cost",
              "efficiency",
              "start_up_cost",
              "shut_down_cost",
              "min_up_time",
              "min_down_time",
#               "ramp_limit_up",
#               "ramp_limit_down"
              ]

# ____

In [ ]:
network = pypsa.Network()

In [ ]:
network.set_snapshots(period)

In [ ]:
if include_fission:
    instance_info += "+fission"

In [ ]:
statedistribution = [0.054, 0.074, 0.126, 0.217, 0.061, 0.075, 0.133, 0.157, 0.103]

## Data Import

In [ ]:
excelFile_mapGermany = pd.ExcelFile(path_import_data + "map_cities.xlsx")
excelFile_mapAbroad = pd.ExcelFile(path_import_data + "map_countries.xlsx")
excelFile_generators_fossil = pd.ExcelFile(path_import_data + "power_grid_mapping_generator_fleet.xlsx")
excelFile_generators_ucData = pd.ExcelFile(path_import_data + "uc_data.xlsx")
excelFile_demand = pd.ExcelFile(path_import_data + "DE Realisierter Stromverbrauch 2017.xlsx")
excelFile_dispatch_ee = pd.ExcelFile(path_import_data+ "DE Realisierte Erzeugung 2017.xlsx")

In [ ]:
df_generators_fossil = pd.read_excel(excelFile_generators_fossil,
                                     sheet_name="fleet_fuel", header=0, usecols="B:D")

df_generators_ucData = pd.read_excel(excelFile_generators_ucData,
                                     sheet_name="2017", header=0, usecols="A:J")
#wind_file = pd.read_excel(path_import_data + "Stromlast und Windeinspeisung.xlsx",
#                          sheet_name="Tabelle1", index_col=0, header=3, usecols="A:F")

df_demand = pd.read_excel(excelFile_demand,
                          sheet_name="mapping_demand", index_col=0, header=0, usecols="A:J")

df_dispatch_wind = pd.read_excel(excelFile_dispatch_ee,
                                 sheet_name="mapping_wind", index_col=0, header=0, usecols="A:C")

df_dispatch_ee = pd.read_excel(excelFile_dispatch_ee,
                               sheet_name="Realisierte Erzeugung", index_col=0, header=0, usecols="A,B,C,E,F,G,L")

In [ ]:
if include_fission:
    df_dispatch_fission = pd.read_excel(excelFile_dispatch_ee,
                                        sheet_name="Realisierte Erzeugung", index_col=0, header=0, usecols="A,H")

    s_dispatch_fission = (df_dispatch_fission.replace('-', 0)
                          .resample('1H').sum()
                          .loc[period]
                          .iloc[:,0])

In [ ]:
df_demand = (df_demand.resample('H').sum()
             .loc[network.snapshots])
df_dispatch_wind = (df_dispatch_wind.resample('H').sum()
                    .loc[network.snapshots])
s_dispatch_ee = (df_dispatch_ee.replace('-',0)
                 .resample('1H').sum()
                 .agg('sum', axis='columns')
                 .loc[period])

## Components

### Setup within Germany

In [ ]:
df_buses = pd.read_excel(excelFile_mapGermany,
                         sheet_name="buses", header=0, usecols="B:D")
df_lines = pd.read_excel(excelFile_mapGermany,
                         sheet_name="lines", header=0, usecols="B:D")

### Setup abroad

In [ ]:
df_countries = pd.read_excel(excelFile_mapAbroad,
                             sheet_name="buses", header=0, usecols="B:D")
df_countries_lines = pd.read_excel(excelFile_mapAbroad,
                                   sheet_name="lines", header=0, usecols="B:D")

df_buses = df_buses.append(df_countries).reset_index(drop=True)
df_lines = df_lines.append(df_countries_lines).reset_index(drop=True)

In [ ]:
pypsa.io.import_components_from_dataframe(network, df_buses, "Bus")

Calculating the length of each line by using the haversine formula (needed to represent the earth's eliptical shape)

In [ ]:
df_lines["length"] = 0
df_lines["x"]=0.0001
df_lines["s_nom"]= 1000000
# factor_coords_to_km = 111

for i in range(len(df_lines)):
    start = [df_buses["x"][df_lines["bus0"][i]],df_buses["y"][df_lines["bus0"][i]]]
    ende = [df_buses["x"][df_lines["bus1"][i]], df_buses["y"][df_lines["bus1"][i]]]
    # Using the haversine forumula to get distance in km
    df_lines.at[i, "length"] = geo.haversine(start, ende)

In [ ]:
pypsa.io.import_components_from_dataframe(network, df_lines, "Line")

### Generators

In [ ]:
df_generators_fossil.rename(columns={"fuel":"carrier",
                           "capacity[MW]":"p_nom"}, inplace=True)
df_generators_fossil["committable"] = True

In [ ]:
df_generators_fossil["bus"] = 0

for j in range(len(df_generators_fossil)):
    # find index of "Bundesland" of each generator
    indices = [i for i, s in enumerate(network.buses.name) if df_generators_fossil["state"][j] in s]
    df_generators_fossil.at[j, "bus"] = indices[0]
    for k in range(len(uc_restrictions)):
        # insert corresponding values according to the unit commitment file to dataframe for import
        df_generators_fossil.at[j, uc_restrictions[k]] = df_generators_ucData[uc_restrictions[k]][df_generators_fossil["carrier"][j]]

In [ ]:
pypsa.io.import_components_from_dataframe(network, df_generators_fossil, "Generator")

### Wind Energy

In [ ]:
df_info_wind = pd.DataFrame({
    'name':['Nordsee', 'Ostsee'],
    'bus':[17, 18],
    'dispatch':[df_dispatch_wind.iloc[:,0], df_dispatch_wind.iloc[:,1]]
})

The nominal power of the generators was set to one because PyPSA internally multiplies p_nom with each value from p_min_pu and p_max_pu to get the dispatched power. Setting p_nom to one offers the possibility to leave the provided data as it is (in MWh).

In [ ]:
for i in range(len(df_info_wind)):
    network.add("Generator",
       name="Offshore {}".format(df_info_wind.loc[i, 'name']),
       bus=df_info_wind.loc[i, 'bus'],
       carrier="Wind",
       p_nom=1,
       marginal_cost=marginal_costs_carrier["Wind"],
       # p_min_pu and p_max_pu have to be the same to ensure fixed amount of dispatched power
       p_min_pu=df_info_wind.loc[i, 'dispatch'],
       p_max_pu=df_info_wind.loc[i, 'dispatch'],
       committable=False) 

Attaching EE-Generation

In [ ]:
for i in range(9):
    tmp = s_dispatch_ee*statedistribution[i]
#     print(tmp)
    network.add("Generator", "EE-Generator {}".format(i),
                bus=i,
                carrier="ee",
                p_nom=1,
                marginal_cost=marginal_costs_carrier["ee"],
                p_min_pu=tmp,
                p_max_pu=tmp,
                committable=False)

Each node abroad gets a capacity of 10 GW to avoid shortages which would result in a crash of the model. Given their high price, they probably won't be used and only become interesting when it comes to future scenarios

In [ ]:
#print(df_countries)

i = 9
for country in df_buses[9:17].values:
    network.add("Generator", name=("Import " + country[0]),
                bus=i,
                carrier="Import",
                p_nom=10000000000,
                marginal_cost=marginal_costs_carrier["Import"],
                committable=False)
    i += 1

Adding nuclear power plants

In [ ]:
if include_fission:
    for i in range(9):
        tmp = s_dispatch_fission*statedistribution[i]
    #     print(tmp)
        network.add("Generator", "Nuclear-Generator {}".format(i),
                    bus=i,
                    carrier="Kernenergie",
                    p_nom=1,
                    marginal_cost=marginal_costs_carrier["Kernenergie"],
                    p_min_pu=tmp,
                    p_max_pu=tmp,
                    committable=False)

### Loads

Convert demand to different periods if necessary

In [ ]:
for i in range(9):
#     demand_data_real = df_demand.iloc[:, i]
    #print(i)
    network.add("Load",
           "myload{0}".format(i),
           bus=i,
           # take datapoints corresponding to timestamps in network.snapshots
           p_set=df_demand.iloc[:,i])

## Linear Optimal Power Flow (incl. unit commitment for generators)

In [ ]:
network.lopf(network.snapshots, solver_name="gurobi")

In [ ]:
print('Objective: ' + '{0:,}'.format(network.objective))

## Exporting for Visualization & Analysis

In [ ]:
path_tmp = path_pickle_data + create_date_format(period) + instance_info + "\\"
if not os.path.exists(path_tmp):
    os.mkdir(path_tmp)

In [ ]:
filename = path_tmp + "network" + ".pickle"

with open(filename, 'wb') as f:
    pickle.dump(network, f)

## Visualization

In [ ]:
gen_outs = network.generators_t.p
# No negative values for plotting
gen_outs[gen_outs < 0] = 0
power_by_carrier = gen_outs.groupby(network.generators.carrier, axis=1).sum()

In [ ]:
graph_colors = {"Erdgas":"orange",
                "Braunkohle":"brown",
                "Steinkohle":"black",
                "Wind":"blue",
                "ee":"green",
                "Kernenergie":"yellow",
                "Import":"red"}

if include_fission:
    col_order = ["Kernenergie", "Wind", "ee", "Braunkohle", "Steinkohle", "Erdgas", "Import"]
else:
    col_order = ["Wind", "ee", "Braunkohle", "Steinkohle", "Erdgas", "Import"]
    
power_by_carrier = power_by_carrier[col_order]

In [ ]:
fig,ax = plt.subplots(1,1)
fig.set_size_inches(30,10)

power_by_carrier.plot(kind="area",ax=ax, grid=True, color=[graph_colors[i] for i in power_by_carrier.columns])

ax.set_xlabel("Date")
ax.set_ylabel("MWh")

In [ ]:
print("Hello World")